# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_curve
import plotly.graph_objects as go
from sklearn.feature_selection import RFE

import plotly.express as px

---
## 2. Load data

In [58]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [59]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [60]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [61]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              14606 non-null  object        
 1   channel_sales                   14606 non-null  object        
 2   cons_12m                        14606 non-null  int64         
 3   cons_gas_12m                    14606 non-null  int64         
 4   cons_last_month                 14606 non-null  int64         
 5   date_activ                      14606 non-null  datetime64[ns]
 6   date_end                        14606 non-null  datetime64[ns]
 7   date_modif_prod                 14606 non-null  datetime64[ns]
 8   date_renewal                    14606 non-null  datetime64[ns]
 9   forecast_cons_12m               14606 non-null  float64       
 10  forecast_cons_year              14606 non-null  int64         
 11  fo

<div class='alert alert-bloc alert-info'>
Besides price diff between december and january at peak, we could look at <b>variations</b> of mid peaks too to complete the analysis.<br>
Also, we could try to calculate the <b>margin divided by consommation</b>. If the ratio is too high, then perhaps we make too much margin on this customer as opposed to competitors. Tihs makes sense as well since we have seen in the EDA part that churn and margins were correlated (though it did not account for client type or size).<br>
Lastly, it woudl be interesting to compare cons_12m and cons_last month. If the mean of the past twelve months is much higher than the consommation last month, then maybe it is a sign that the customer attempts to reduce his consommation and change supplier. We could create several features (like is it slightly higher, meaning a ratio of 1:1.1 or much higher like a ratio of 1:2). These would be <b>binary features</b>. If it is a seasonal effect, then it would not have much predictive power (since all would be 1). If it is not, then it might be a good indicator of customer habits change.
</div>

In [63]:
df_results = pd.merge(df, diff, how='left', on='id')
# df_results.info()

---

### Variations mid peak

In [64]:
price_df.head(2)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0


In [65]:
# Group mid-peak prices by companies and month
monthly_mid_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_mid_peak_var': 'mean', 'price_mid_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_mid_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_mid_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_mid_peak_var': 'dec_1', 'price_mid_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['midpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_mid_peak_var']
diff['midpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_mid_peak_fix']
diff = diff[['id', 'midpeak_diff_dec_january_energy','midpeak_diff_dec_january_power']]
# diff.head()

In [66]:
df_results = pd.merge(df_results, diff, how='left', on='id')
# df_results.info()

---

### Ratio margin_gross_pow_ele / cons_last_month

In [67]:
print(df.columns)

Index(['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_p

In [68]:
df_results['Ratio_margin_per_cons'] = df_results['margin_gross_pow_ele'] / df_results['cons_last_month']
df_results['Ratio_margin_per_cons'] = df_results['Ratio_margin_per_cons'].fillna(0)

---

### Trends of energy consumption variations

<div class='alert alert-bloc alert-info'> As precised in the earlier bloc, we will create three binary features allowing us to compare the consumption of power in the last month with the consumption of power in the last twelve months.<br>
We will thus create three binary features which will be: <b>cons_last_month > ratio * cons_12m</b> with ratio being equal to 1.25, 1.5 and 2 respectively.<br>
Alertnately, we could have just created the feature cons_ratio which would have been cons_last_month / cons_12m, but due to the large variability of consumption, I prioritized binary features to keep interpretability, robusteness to outliers, and non-linear patterns. I am aware that those ratio might have to be tuned with the help of experts to define them more rigorously though.</div>

In [69]:
# RATIO VARIBALES
RATIO_1 = 1.25 / 12
RATIO_2 = 1.5 / 12
RATIO_3 = 2.0 / 12

In [70]:
df_results['Ratio_pow_1'] = df_results['cons_last_month'] > RATIO_1 * df_results['cons_12m']
df_results['Ratio_pow_2'] = df_results['cons_last_month'] > RATIO_2 * df_results['cons_12m']
df_results['Ratio_pow_3'] = df_results['cons_last_month'] > RATIO_3 * df_results['cons_12m']

In [71]:
# Type fixing
df_results['Ratio_pow_1'] = df_results['Ratio_pow_1'].astype(int)
df_results['Ratio_pow_2'] = df_results['Ratio_pow_2'].astype(int)
df_results['Ratio_pow_3'] = df_results['Ratio_pow_3'].astype(int)

---
### Cleaning

In [72]:
df_results['Ratio_margin_per_cons'] = df_results['Ratio_margin_per_cons'].replace(float('inf'), 0)
df_results = pd.get_dummies(
    df_results,
    columns=['channel_sales'],       # la colonne à encoder
    prefix='cs_',          # préfixe pour les nouvelles colonnes
    drop_first=True,              # True pour éviter la colinéarité (k-1 colonnes)
    dtype=int
)

df_results = pd.get_dummies(
    df_results,
    columns=['origin_up'],       # la colonne à encoder
    prefix='ou_',          # préfixe pour les nouvelles colonnes
    drop_first=True,              # True pour éviter la colinéarité (k-1 colonnes)
    dtype=int
)

df_results['date_activ'] = pd.to_datetime(df_results['date_activ'], format='%Y-%m-%d')
df_results['activ_year']  = df_results['date_activ'].dt.year
df_results['acitv_month'] = df_results['date_activ'].dt.month
df_results['activ_day']   = df_results['date_activ'].dt.day

df_results['date_end'] = pd.to_datetime(df_results['date_end'], format='%Y-%m-%d')
df_results['end_year']  = df_results['date_end'].dt.year
df_results['end_month'] = df_results['date_end'].dt.month
df_results['end_day']   = df_results['date_end'].dt.day

df_results['date_renewal'] = pd.to_datetime(df_results['date_renewal'], format='%Y-%m-%d')
df_results['renewal_year']  = df_results['date_renewal'].dt.year
df_results['renewal_month'] = df_results['date_renewal'].dt.month
df_results['renewal_day']   = df_results['date_renewal'].dt.day

df_results = df_results.drop(columns=['date_activ', 'date_end', 'date_modif_prod', 'date_renewal'])

df_results['has_gas'] = df_results['has_gas'].replace({'t': 1, 'f': 0})

df_results.head(2)

C:\Users\Louis\AppData\Local\Temp\ipykernel_3220\2229628116.py:35: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,ou__usapbepcfoloekilkwsdiboslwaxobdp,activ_year,acitv_month,activ_day,end_year,end_month,end_day,renewal_year,renewal_month,renewal_day
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,...,0,2013,6,15,2016,6,15,2015,6,23
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,...,0,2009,8,21,2016,8,30,2015,8,31


<div class='alert alert-bloc alert-danger'>To exploit the data, we will need to remove the id columns. As for the non-integer columns, we need label encode them. Since we do not want to introduce an order relation in the channel sales, I will use <b>one-hot-encoder</b> for this feature. For the dates, I will turn them in three columns (year|month|day) : For instance, for the feature 'date_activ', I will create 'date_activ_yyyy', 'date_activ_mm', 'date_activ_dd' and drop the original date column.</div>

---

## 4. Recursive Feature Extraction

<div class='alert alert-bloc alert-info'>We successfully added some high-quality features in the previous steps. However, our dataset also happens to be filled with a lot of low-value features. We believe that removing the features that have little to no predictive power is likely to improve our model's performance. To do so, we will use an RFE (<b>Recursive Feature Extraction</b>) to remove the most impure features on our random forest model.<br>
Note: To verify that we did not mistakenly remove high-value information, we will compare the model before and after this step and compare them on a metric. Since we want to accurately identify churn, we should not rely on accuracy alone (otherwise it might lead to high metric values even if we do not predict anything). Instead, we will train our models on the <b>ROC-AUC metric</b>.<div>

---

### Before RFE

In [73]:
X = df_results.drop(columns=['churn', 'id'])
y = df_results['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)
scaler = StandardScaler()


weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y_train)
class_weights = {cls: wt for cls, wt in zip(np.unique(y_train), weights)}
print(class_weights)

scaler.fit_transform(X_train, y_train)
scaler.transform(X_test)

model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=7, class_weight=class_weights)
model.fit(X_train, y_train)


y_proba = model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_proba)
print(f"Test-set ROC-AUC with class weights: {auc:.3f}")

{0: 0.5537965683951085, 1: 5.147136563876652}
Test-set ROC-AUC with class weights: 0.659


In [74]:
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC Curve'))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Classifier', line=dict(dash='dash')))

fig.update_layout(
    title='Receiver Operating Characteristic (ROC) Curve',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    xaxis=dict(constrain='domain'),
    yaxis=dict(scaleanchor='x', scaleratio=1),
    legend=dict(x=0.7, y=0.1)
)

fig.show()

--- 

### Recursive Feature Extraction

In [76]:
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression()

selector = RFE(estimator=estimator, n_features_to_select=30, step=1)
selector.fit(X_train, y_train)

ranking_df = pd.DataFrame({'feature': X_train.columns, 'ranking': selector.ranking_})
selected = ranking_df[ranking_df['ranking'] == 1]['feature']
if hasattr(selector.estimator_, 'coef_'):
    importances = dict(zip(selected, abs(selector.estimator_.coef_[0])))
elif hasattr(selector.estimator_, 'feature_importances_'):
    importances = dict(zip(selected, selector.estimator_.feature_importances_))
importance_df = pd.DataFrame({'feature': list(importances.keys()), 'importance': list(importances.values())})
importance_df = importance_df.sort_values('importance', ascending=False)

fig = px.bar(importance_df, x='feature', y='importance')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

In [79]:
X = df_results.drop(columns=['id', 'churn'])
y = df_results['churn']
selected_features = X.columns[selector.support_]
X_selected = X[selected_features]

X_train_selected, X_test_selected, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=7
)

In [80]:
scaler = StandardScaler()


weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y_train)
class_weights = {cls: wt for cls, wt in zip(np.unique(y_train), weights)}
print(class_weights)

scaler.fit_transform(X_train_selected, y_train)
scaler.transform(X_test_selected)

model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=7, class_weight=class_weights)
model.fit(X_train_selected, y_train)


y_proba = model.predict_proba(X_test_selected)[:, 1]

auc = roc_auc_score(y_test, y_proba)
print(f"Test-set ROC-AUC with class weights: {auc:.3f}")

{0: 0.5540591805766313, 1: 5.124561403508772}
Test-set ROC-AUC with class weights: 0.668


In [81]:
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC Curve'))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Classifier', line=dict(dash='dash')))

fig.update_layout(
    title='Receiver Operating Characteristic (ROC) Curve',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    xaxis=dict(constrain='domain'),
    yaxis=dict(scaleanchor='x', scaleratio=1),
    legend=dict(x=0.7, y=0.1)
)

fig.show()

<div class='alert alert-bloc alert-info'>As expected, selecting features through RFE did increase performances slightly. Additionally, it looks like the channel sales (as seen in the EDA) show remarkable predictive power.
Although price difference between january and december (peak and mid) show predictive power, it is insufficient to conclude on its own, as it exhibits lower feature importance compared to the engineered features such as 'year' or the custom ratios that we made.</div>